In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
file ='BlackFriday.xlsx'
bf = pd.read_excel(file)
bf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 12 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null int64
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
Purchase                      537577 non-null int64
dtypes: float64(2), int64(6), object(4)
memory usage: 49.2+ MB


In [2]:
bf.isnull().sum()


User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            166986
Product_Category_3            373299
Purchase                           0
dtype: int64

In [3]:
bf.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4,0,8,NaN,NaN,7969
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871


In [5]:
bf['Product_Category_2']= (bf['Product_Category_2'].fillna(0))
bf['Product_Category_3']= (bf['Product_Category_3'].fillna(0))


In [6]:
bf.isnull().sum()


User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Purchase                      0
dtype: int64

In [7]:
bf['Age'].value_counts()


26-35    214690
36-45    107499
18-25     97634
46-50     44526
51-55     37618
55+       20903
0-17      14707
Name: Age, dtype: int64

In [8]:
bf['City_Category'].value_counts()


B    226493
C    166446
A    144638
Name: City_Category, dtype: int64

In [9]:
bf['Gender'].value_counts()


M    405380
F    132197
Name: Gender, dtype: int64

In [13]:
City_dummies = pd.get_dummies((bf['City_Category']))


In [14]:
Age_dummies = pd.get_dummies((bf['Age']),drop_first = True)


In [15]:
bf_with_dummies = pd.concat([bf.iloc[:,:],
                      City_dummies,
                      Age_dummies],
                      axis = 1)

In [16]:
bf_with_dummies = bf_with_dummies.drop(['City_Category','Age'],axis=1)


In [17]:
bf_with_dummies.loc[:, "Gender"] = pd.factorize(bf_with_dummies.Gender)[0]# suna bak 


In [19]:
bf_with_dummies.head()

,User_ID,Product_ID,Gender,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,A,B,C,18-25,26-35,36-45,46-50,51-55,55+
0,1000001,P00069042,0,10,2,0,3,0.0,0.0,8370,1,0,0,0,0,0,0,0,0
1,1000001,P00248942,0,10,2,0,1,6.0,14.0,15200,1,0,0,0,0,0,0,0,0
2,1000001,P00087842,0,10,2,0,12,0.0,0.0,1422,1,0,0,0,0,0,0,0,0
3,1000001,P00085442,0,10,2,0,12,14.0,0.0,1057,1,0,0,0,0,0,0,0,0
4,1000002,P00285442,1,16,4,0,8,0.0,0.0,7969,0,0,1,0,0,0,0,0,1


In [20]:
y = bf_with_dummies.loc[:, "Purchase"]
X = bf_with_dummies.drop(['Purchase', 'User_ID', 'Product_ID'],axis = 1)


In [21]:
X.head()

,Gender,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,A,B,C,18-25,26-35,36-45,46-50,51-55,55+
0,0,10,2,0,3,0.0,0.0,1,0,0,0,0,0,0,0,0
1,0,10,2,0,1,6.0,14.0,1,0,0,0,0,0,0,0,0
2,0,10,2,0,12,0.0,0.0,1,0,0,0,0,0,0,0,0
3,0,10,2,0,12,14.0,0.0,1,0,0,0,0,0,0,0,0
4,1,16,4,0,8,0.0,0.0,0,0,1,0,0,0,0,0,1


In [22]:
y.head()

0     8370
1    15200
2     1422
3     1057
4     7969
Name: Purchase, dtype: int64

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X,
                                    y,
                                    test_size = 0.2,
                                    random_state = 508,
                                    stratify = X['Gender'])


In [25]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
predictions = lin_reg.predict(X_test)


In [27]:
r2_score = r2_score(y_test, predictions) 
mean_squareed_error_score = mean_squared_error(y_test, predictions) 
exp_var_score =explained_variance_score (y_test, predictions)

In [28]:
print(r2_score)

0.1350796187770512


In [29]:
print(mean_squareed_error_score)

21427539.99487691


In [30]:
print(exp_var_score)

0.13507961890025377


In [33]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]
forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)


grid_search.fit(X_train, y_train)
grid_search.best_params_
grid_search.best_estimator_
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

3409.70877943897 {'max_features': 2, 'n_estimators': 3}
3255.865414633062 {'max_features': 2, 'n_estimators': 10}
3199.4453782260794 {'max_features': 2, 'n_estimators': 30}
3281.725284445917 {'max_features': 4, 'n_estimators': 3}
3142.8085140462226 {'max_features': 4, 'n_estimators': 10}
3105.319537796705 {'max_features': 4, 'n_estimators': 30}
3238.6334012087805 {'max_features': 6, 'n_estimators': 3}
3115.8839210650412 {'max_features': 6, 'n_estimators': 10}
3079.325827846493 {'max_features': 6, 'n_estimators': 30}
3229.887476726879 {'max_features': 8, 'n_estimators': 3}
3114.096329456567 {'max_features': 8, 'n_estimators': 10}
3078.967509754481 {'max_features': 8, 'n_estimators': 30}
3400.933040694163 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
3334.13391225926 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
3338.147433880526 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
3274.8204223138664 {'bootstrap': False, 'max_features': 3, 'n_estimators':

In [38]:
final_model = grid_search.best_estimator_


In [90]:
random_forest_predictions = final_model.predict(X_test)


In [100]:
exp_var_score_random_forest =explained_variance_score (y_test, final_predictions)
print(exp_var_score_random_forest)

0.6246013008467235


In [101]:
mean_squareed_error_score_randomforest= mean_squared_error(y_test, final_predictions) 
print(mean_squareed_error_score_randomforest)

9300192.556964684
